# CSG CAD in Python

## Setup
To setup this environment, you need to install:
* [OpenSCAD](https://www.openscad.org/downloads.html)

Then you need to open your anaconda command prompt and power shell.
* install [SolidPython](https://github.com/SolidCode/SolidPython#using-solidpython) using the command `pip install solidpython`
* install [viewscad](https://github.com/nickc92/ViewSCAD) by running the command `pip install viewscad

Alternatively you can run the lines bellow`

In [ ]:
#@title <-- (click this) Install all the software this notebook need. Then hit restart runtime like it says
# First we install openscad
!sudo add-apt-repository ppa:openscad/releases -y
#!sudo apt-get update
!sudo apt-get install openscad
# Now we install Viewscad

!pip install viewscad
!pip install solidpython

In [ ]:
from solid import *
from solid.utils import *
import viewscad

r = viewscad.Renderer()

from google.colab import output
output.enable_custom_widget_manager()

Above we setup the environment. It includes importing the relivant functions and telling the rendered where openscad is. You may need to change the line `openscad_exec='C:\Program Files\OpenSCAD\openscad.exe'`

**Next lets generate a simple cylinder**

In [ ]:
c = cylinder(r=10,h=5)
r.render(c)
print(scad_render(c))

In [ ]:
print(c)

The code is converting the cylinder to a very basic form of an equation into a triangulated surfce. But as we can see the resolution isnt great. This is the same problem we have when we convert from a nice geoemtry in Solidworks/Autodesk to STL files.

How can we fix it? We increase the discritization resolution

In [ ]:
c = cylinder(r=10,h=5,segments=1000)
r.render(c)

Ok, so openSCAD aaaaaaaaaaalways converts from CSG into a mesh to be viewed. We can see this when we zoom into the objet and all of the sudden there isnt anything inside! And we can set the resolution of this conversion.

## What are the primitives?
### sphere

In [ ]:
# Sphere with radius 10
s1 = sphere(r=10)
r.render(s1)

In [ ]:
# Set diameter to 5
s2 = sphere(d=5, segments = 100)
r.render(s2)

### Cuboids

In [ ]:
c1 = cube(10)
r.render(c1)

The basic function puts a corner at the origin with the side length you set.
We can also tell it to automatically put it with the center at the origin

In [ ]:
c2 = cube(10,True)
r.render(c2)

We can also use it to make a rectangular prisim.

In [ ]:
c3 = cube([10,20,30])
r.render(c3)

You can note that there is no segments argument to cube because a triangulated mesh is a perfect representation of a cube.

### Cylinder
we saw the basics above, but we can also make Cones using cylinder

In [ ]:
c2 = cylinder(r1 = 10, r2=5,h=5)
r.render(c2)

It is also possible to define cone sections using diameters and to shift the center to the center of the height axis. The height axis will be in the Z direction

In [ ]:
c3 = cylinder(d1 = 10, d2=5,h=5,center=True)
r.render(c3)

### Polyhedron

What if those primitives are not what you need? What if I need to make a pyramids?  or Tetra hedra, or any of the platonic solids besides a cube?

In [ ]:
points = [[0,0,0],[0,1,0],[1,1,0],[1,0,0],[0.5,0.5,1]]
faces = [[0,1,2,3], # Base
         [0,1,4],[1,2,4],[2,3,4],[3,0,4]]
p = polyhedron(points = points,faces = faces)
r.render(p)

In [ ]:
points=[ [10,10,0],[10,-10,0],[-10,-10,0],[-10,10,0], # the four points at base
           [0,0,10]  ]                               # the apex point
faces=[ [0,1,4],[1,2,4],[2,3,4],[3,0,4],              # each triangle side
              [1,0,3],[2,1,3] ]                       # two triangles for square base
p2 = polyhedron(points = points,faces = faces)
r.render(p2)

In [ ]:
def pyramid(side,height):
    s = side/2.0
    points=[ [s,s,0],[s,-s,0],[-s,-s,0],[-s,s,0], # the four points at base
           [0,0,height]  ]                               # the apex point
    faces=[ [0,1,4],[1,2,4],[2,3,4],[3,0,4],              # each triangle side
              [1,0,3],[2,1,3] ]                       # two triangles for square base
    return polyhedron(points = points,faces = faces)
r.render(pyramid(10,10))

If your polygon is having issues with faces not showing up. look at the segment on debugging your faces [here](https://en.wikibooks.org/wiki/OpenSCAD_User_Manual/Primitive_Solids#polyhedron)

### Well what about those basic CSG operations?
* Union
* Difference
* Intersection

Arent they just the opperations of a **SET??**

In [ ]:
c3 = c1 + c2
r.render(c3)

In [ ]:
c3 = c1 - c2
r.render(c3)

In [ ]:
c3 = c2 - c1
r.render(c3)

In [ ]:
c3 = c1*c2
r.render(c3)

### Hole

In [ ]:
pipe_od = 10
pipe_id = 3
seg_length = 10
outer = cylinder(r=pipe_od, h=seg_length)
inner = cylinder(r=pipe_id, h=seg_length)
pipe_a = outer - hole()(inner)
r.render(pipe_a)

In [ ]:
pipe_a = outer + hole()(inner)
r.render(pipe_a)

## Transformations

* Scale
    * scale a
    * scale [a1,a2,a3]
    * resize [x,y,z]
* Rotate
    * axis angle
    * [dx,dy,dz] euler
* Translate
    * tx ty tz
* mirror
    * plane normal n1 n2 n3
* Homogenous SE(3)
    


### Scaling

In [ ]:
x = scale(1.5)(cube(10))-cube(10)
r.render(x)

In [ ]:
x = scale([1,2,3])(sphere(10))
r.render(x)

In [ ]:
x = resize([20,5,10])(sphere(10))
r.render(x)

## Rotation SO(3)

As we discussed in class there are several ways to do rotations. namely Axis-angle, Euler Angles, and Quaternions. Natively OpenSCAD only does the first two. Lets look at Axis-angle

In [ ]:
angle = 45
axis = [0,1,0]
x = rotate(angle,axis)(pyramid(10,10))
r.render(x)

In [ ]:
angle = 45
axis = [1,1,0]
x = rotate(angle,axis)(pyramid(10,10))
r.render(x)

In [ ]:
angles = [20,10,5]
x = rotate(angles)(pyramid(10,10))
r.render(x)

In [ ]:
angles = [20,10,5]
x2 = rotate(angles[0],[1,0,0])(pyramid(10,10))
r.render(x2)

In [ ]:
x2 = rotate(angles[1],[0,1,0])(x2)
r.render(x2)

In [ ]:
x2 = rotate(angles[2],[0,0,1])(x2)
r.render(x2)

In [ ]:
r.render(x-x2+sphere(1))

This means the system uses the XYZ euler angles. NOT any of the implicit XY'Z'' or the standard ZXZ system. So be aware when someone quotes you euler angles!

We can also use quaterions, but we need to insert our own transformation. To do that we can convert to Matrix form using the equation below

$$ R= \left(\begin{matrix}1-2(q_j^2-q_k^2)&2(q_iq_j-q_kq_r)&2(q_iq_k+q_jq_r)\\2(q_iq_j+q_kq_r)&1-2(q_i^2-q_k^2)&2(q_jq_k-q_iq_r)\\2(q_iq_k-q_jq_r)&2(q_jq_k+q_iq_r)&1-2(q_i^2-q_j^2)\end{matrix}\right) $$


We should note that the naming here is carefully chosen. some people list quaternions as $$q = [q_r,q_i,q_j,q_k]$$ others use $$q = [q_i,q_j,q_k,q_r]$$ so be careful when storing quaternion as a list or connecting two bits of quaternion code from different sources together. Scipy and numpy use the second convention


In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as RM
rm = RM.from_quat([np.sin(np.pi/4),0, 0, np.cos(np.pi/4)])
rot = rm.as_dcm()
print(rot)
y = multmatrix(rot)(pyramid(10,10))
r.render(y)

## Translate T(3)

In [ ]:
y = translate([0,10,0])(pyramid(10,10))+pyramid(10,10)
r.render(y)

### Mirror

In [ ]:
c = cube([30,20,10])
c1 = translate([20,0,0])(rotate(45,[0,0,1])(c))
c2 = mirror([1,1,0])(c1)
r.render(c1+c2)


## SE(3)

In [ ]:
import numpy as np
def Nx(n):
    x = n[0]
    y = n[1]
    z = n[2]
    nx = np.array( [ [0, -z,  y],
                     [z,  0,  x],
                     [-y,  x, 0] ] )
    return nx

def AxisAngle(n,theta):
    mag = np.linalg.norm(n)
    v = n/mag
    s = np.sin(theta)
    c = np.cos(theta)
    NM = Nx(v)
    r = np.eye(3)+s*NM+(1-c)*NM@NM
    return r

def AxisAngleDistance(n,theta,D):
    R = AxisAngle(n,theta)
    d = np.array(D).T.reshape((1,3))
    return np.block([
                    [R , d.T],
                    [0,0,0, 1]
                    ])

In [ ]:
n = [0,0,1]
theta = np.pi/4
D = [3,1,0]
RD = AxisAngleDistance(n,theta,D)

y = multmatrix(RD)(pyramid(10,10))
r.render(y)

### Other Operations
* [minkowski sum](https://en.wikibooks.org/wiki/OpenSCAD_User_Manual/Transformations)
* [Convex Hull](https://en.wikibooks.org/wiki/OpenSCAD_User_Manual/Transformations)
* Shear transforms using m Matrix
* [Affine Transforms](https://www.wikiwand.com/en/Affine_transformation)

In [ ]:
x = cube([10,10,1])+cylinder(1,True)
r.render(x)

In [ ]:
x = minkowski()(cube([10,10,1]),cylinder(1,True))
r.render(x)

In [ ]:
q = sphere(1)
x = minkowski()(pyramid(10,10),translate([5,5,0])(q))
r.render(x)

In [ ]:
q = sphere(1)
x = pyramid(10,10)+translate([5,5,0])(q)
r.render(x)

## Convex Hull

In [ ]:
c1 = cylinder(1,1)
c2 = translate([10,10,0])(c1)
y = c1+c2
r.render(y)

In [ ]:
c1 = cylinder(1,1)
c2 = translate([10,10,0])(c1)
y = hull()(c1,c2)
r.render(y)

In [ ]:
c1 = cylinder(1,1)
c2 = translate([10,10,0])(c1)
c3 = translate([0,10,0])(c1)
y = hull()(c1,c2,c3)
r.render(y)

In [ ]:
c1 = cylinder(1,1)
c2 = translate([10,10,0])(c1)
c3 = translate([0,10,0])(c1)
y = c1+c2+c3
r.render(y)

In [ ]:
c1 = cylinder(1,1)
c2 = translate([10,10,0])(c1)
c3 = translate([0,10,0])(c1)
c4 = translate([4,7,0])(c1)
y = c1+c2+c3+c4
r.render(y)

In [ ]:
c1 = cylinder(1,1)
c2 = translate([10,10,0])(c1)
c3 = translate([0,10,0])(c1)
c4 = translate([4,7,0])(c1)
y = hull()(c1, c2, c3, c4)
r.render(y)

## 2D operations
* square
* circle
* polygon
* text

In [ ]:
s1 = square(size = [4,10],center = True)
s2 = square(size = 1.5, center = False)
y = s1-s2
print(scad_render(y))

In [ ]:
r.render(y)

In [ ]:
c1 = circle(r=10)
c2 = circle(d=30)
y = c2-c1
print(scad_render(y))

In [ ]:
r_c = 30
n = 6
xs = [r_c*np.cos(2*np.pi/n*i)for i in range(0,n+1)]
ys = [r_c*np.sin(2*np.pi/n*i)for i in range(0,n+1)]
pts = list(zip(xs,ys))
y = polygon(points=pts)
print(scad_render(y))

In [ ]:
y= text('ME480')
print(scad_render(y))

## 2D to 3D
* linear extrude
    * text to use to make labels
    * twisting
* rotation extrude
    * [solids of rotations ](https://en.wikibooks.org/wiki/OpenSCAD_User_Manual/2D_to_3D_Extrusion#Linear_Extrude)
   

In [ ]:
y = linear_extrude(height =10,center=True)(text('ME480'))
r.render(y)

In [ ]:
c1 = circle(r=10)
c2 = circle(d=30)
y = linear_extrude(height =10,center=True)(c2-c1)
r.render(y)

In [ ]:
y = linear_extrude(height=10, twist=90)(text('ME480'))
angle = 45
axis = [1,1,0]
x = rotate(angle,axis)(y)
r.render(x)

In [ ]:
y = linear_extrude(height=10, twist=90,slices = 100)(text('ME480'))
r.render(y)

In [ ]:
c1 = translate([0,30,0])(circle(r=10))
c2 = translate([0,30,0])(circle(d=30))
x = c2-c1
y = linear_extrude(height=30, twist=90)(x)
r.render(y)

In [ ]:
c1 = translate([0,30,0])(circle(r=10))
c2 = translate([0,30,0])(circle(d=30))
x = c2-c1
y = linear_extrude(height=300, twist=720)(x)
r.render(y)

In [ ]:
r_c = 20
n = 4
l = 10*r_c
xs = [r_c*np.cos(2*np.pi/n*i)for i in range(0,n+1)]
ys = [r_c*np.sin(2*np.pi/n*i)for i in range(0,n+1)]
pts = list(zip(xs,ys))
y = polygon(points=pts)
z = linear_extrude(height=l, twist=4*360,slices = 100)(y)
r.render(z)

In [ ]:
r_c = 1
n = 4
l = 10*r_c
xs = [r_c*np.cos(2*np.pi/n*i)for i in range(0,n+1)]
ys = [r_c*np.sin(2*np.pi/n*i)for i in range(0,n+1)]
pts = list(zip(xs,ys))
y = polygon(points=pts)
z = linear_extrude(height=l, twist=4*360,slices = 100,scale = 0.5)(y)
r.render(z)

In [ ]:
z = linear_extrude(height=10, twist=4*360,slices = 100, scale=[1,2])(y)
r.render(z)

In [ ]:
z = rotate_extrude(angle = 360)(translate([2,0,0])(y))
print(scad_render(z))
r.render(z)

In [ ]:
print(scad_render(y))

In [ ]:
z = rotate_extrude(angle = 360,segments=100)(translate([2,0,0])(y))
print(scad_render(z))
r.render(z)

In [ ]:
scad_render(z)

In [ ]:
def exportToScad(solidObj,file,path = "."):
    location = path+'/'+file+".scad"
    with open(location,'w') as f:
        print(f.write(scad_render(solidObj)))

exportToScad(cube(10),"test")

In [ ]:
exportToScad(z,"Ztest")

## Color

In [ ]:
transparent_blue = color([0,0,1, 0.5])(translate([0,10,0])(cube(10)))  # Specify with RGB[A]
red_obj = color(Red)(cube(10))                    # Or use predefined colors
print(scad_render(transparent_blue+red_obj))

In [ ]:
r.render(red_obj)

![image.png](attachment:image.png)

## Imports
* stl import
* import SCAD
    * lego Brick Model

In [ ]:
q = import_stl("C://Users//jil26//Documents//GitHub//ME480//JupyterNotebooks//darthVader.stl")
w = translate([100,0,0])(q)
print(scad_render(q+w))#.replace('\n','').replace('\t','')
r.render(q)

Sometimes the renderer isnt perfect, so you have to copy and paste into openscad

# [OpenSCAD MCAD Library](https://github.com/openscad/MCAD)

In [ ]:
scadfile = import_scad('C://Users//jil26//Documents//GitHub//MCAD//lego_compatibility.scad')


#print(scadfile)
width = 10
length = 5
height = 2
b = scadfile.block(width,length,height,reinforcement=True)
r.render(b)

In [ ]:
width = 10
length = 5
height = 2
b = scadfile.block(width,length,height)
r.render(b)

In [ ]:
x = minkowski()(cylinder(r=5,h=5),sphere(1))
cn =cylinder(r=10,h=5)- hole()(cylinder(r=5,h=5))-translate([0,0,5])(x)
r.render(cn)